<a href="https://colab.research.google.com/github/vishvas-chauhan/First_Repository/blob/master/regression_multiplemodel_gridsearchcv_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import numpy as np

In [7]:
from google.colab import files
f1=files.upload()

Saving webstore sales - webstore_col.csv to webstore sales - webstore_col.csv


In [64]:
df=pd.read_csv(r'webstore sales - webstore_col.csv')
#df.apply(lambda x:len(x.unique()))


In [65]:
df.nunique()

Product name                 567
Product ID                   511
HOUR Date                   3370
Order                       4917
Order Billing City          2176
Order Billing Country         70
Order Discount code           27
Refund note                  186
Customer Full name          4247
Total quantity sold            9
Total orders                   2
Total gross sales            356
Total discounts              154
Total refunds                154
Total net sales              567
Total taxes                  320
Total sales                  357
Total cost of goods sold     744
Total gross margin          1321
Image                        585
month                         12
week                          53
week name                      7
year                           3
hour                          24
Refund Y/N                     2
Returned                       2
Product Category 1            11
Product Category 2             6
Product Category 3             7
Product Ca

In [66]:
df=df[['Product Category 5','year','month','Order Billing Country','Order Billing City','Total net sales']]

In [ ]:
#q_low = df["Total quantity sold"].quantile(0.01)
#q_hi  = df["Total quantity sold"].quantile(0.99)

#df= df[(df["Total quantity sold"] < q_hi) & (df["Total quantity sold"] > q_low)]

In [67]:
df=df.groupby(['Product Category 5','month','year','Order Billing Country','Order Billing City'])['Total net sales'].sum().reset_index()

In [68]:
df.shape

(4591, 6)

In [69]:
df.year.value_counts()

2020    3060
2021    1007
2019     524
Name: year, dtype: int64

In [70]:
df=df[df['year']!=2019]
df=df[df['year']!=2020]

In [71]:
df=df[df['Total net sales']!=0]
df=df[df['Product Category 5']!='Wrong Code']
#df=df[df['Total net sales']<=0]

In [72]:
df['Product Category 5'].value_counts()

TOP SECOND SKIN       343
NIGHT & UNDERWEAR     151
LEGGING               124
HATS & CAPS            48
ALL-IN-ONES            45
EYEWEAR                37
PANT                   26
TOP KNIT               20
MASK                   20
TOP TSHIRT             19
JACKET & BLAZER        15
TOP OTHER              10
HEADBAND                9
GLOVE                   9
TOP BLOUSE & SHIRT      8
COAT                    7
DRESS                   7
NECKLACE & CHOKER       6
BOOTS ANKLE             5
SHORT                   4
SLIDE                   4
EARRING                 3
SKIRT                   3
SOCK                    2
BOOTS KNEE-HIGH         2
PUMP                    1
RING                    1
SNEAKER                 1
BODY                    1
Name: Product Category 5, dtype: int64

In [73]:
df.head()

,Product Category 5,month,year,Order Billing Country,Order Billing City,Total net sales
1,ALL-IN-ONES,1,2021,Germany,Berlin,366.67
2,ALL-IN-ONES,1,2021,Germany,Hamburg,366.67
3,ALL-IN-ONES,1,2021,Germany,Schwabsoien,366.67
4,ALL-IN-ONES,1,2021,Germany,Winterhausen,366.67
6,ALL-IN-ONES,1,2021,Mexico,Mexico,374.00


In [74]:
from sklearn.model_selection import train_test_split

In [75]:
X=df[['Product Category 5','month','year','Order Billing Country','Order Billing City']]
y=df[['Total net sales']]

In [76]:
X['month']=X['month'].astype(float)
X['year']=X['year'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [77]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.90, random_state =42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.85, random_state =42)
categorical_features_indices = np.where(X.dtypes != np.float)[0]
categorical_features_indices

array([0, 3, 4])

In [37]:
!pip install catboost 

     |████████████████████████████████| 65.7MB 57kB/s 


In [78]:
import catboost
from catboost import MetricVisualizer, Pool, CatBoostRegressor, cv
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [79]:
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()


In [80]:
X_train=X_train.apply(LabelEncoder().fit_transform)
y_train=y_train.apply(LabelEncoder().fit_transform)

In [59]:
#X_train=X_train.apply(LabelEncoder().inverse_transform)

In [81]:
X_train

,Product Category 5,month,year,Order Billing Country,Order Billing City
1854,25,0,0,42,195
40,0,1,0,43,375
1938,25,1,0,3,434
1944,25,1,0,7,305
1297,14,2,0,43,64
...,...,...,...,...,...
1982,25,1,0,24,1
400,6,1,0,22,185
1192,14,0,0,43,143
475,8,1,0,19,365


In [95]:
model_params = {
    'svr':{'model':SVR(gamma='auto'),'params':{'C':[1,10],'kernel':['rbf']}},
    'rfr':{'model':RandomForestRegressor(),'params':{'max_depth':[2,5]}},
    #'lr': {'model':LinearRegression()},
    'cbr':{'model':CatBoostRegressor(),'params':{'iterations':[150,300],'loss_function': ['RMSE']}},
}

In [ ]:
 #'best_score': clf.best_score_,

In [99]:
scores=[]

for model_name, mp in model_params.items():
  clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
  clf.fit(X_train,y_train)
  scores.append({'model':model_name,'best_score': clf.best_score_,'best_params':clf.best_params_})

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Learning rate set to 0.149226
0:	learn: 41.7703867	total: 840us	remaining: 125ms
1:	learn: 40.5242725	total: 1.68ms	remaining: 124ms
2:	learn: 40.0424477	total: 2.58ms	remaining: 126ms
3:	learn: 39.4600188	total: 3.56ms	remaining: 130ms
4:	learn: 37.9373516	total: 4.45ms	remaining: 129ms
5:	learn: 37.6037977	total: 5.37ms	remaining: 129ms
6:	learn: 37.1521712	total: 6.19ms	remaining: 126ms
7:	learn: 36.6406191	total: 7.09ms	remaining: 126ms
8:	learn: 36.3713935	total: 7.94ms	remaining: 124ms
9:	learn: 36.0540134	total: 8.86ms	remaining: 124ms
10:	learn: 35.7664198	total: 9.68ms	remaining: 122ms
11:	learn: 35.1533313	total: 10.5ms	remaining: 121ms
12:	learn: 34.9812520	total: 11.5ms	remaining: 121ms
13:	learn: 34.8149372	total: 12.3ms	remaining: 120ms
14:	learn: 34.7169096	total: 13.2ms	remaining: 119ms
15:	learn: 34.6162816	total: 14.1ms	remaining: 118ms
16:	learn: 34.5284691	total: 14.8ms	remaining: 116ms
17:	learn: 34.1263848	total: 15.7ms	remaining: 115ms
18:	learn: 33.8388284	total

In [100]:
df2=pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [101]:
df2

,model,best_score,best_params
0,svr,-0.011421,"{'C': 10, 'kernel': 'rbf'}"
1,rfr,0.377007,{'max_depth': 5}
2,cbr,-207.123609,"{'iterations': 150, 'loss_function': 'RMSE'}"


In [ ]:
dfdssd

In [102]:
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)

val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_features_indices)

# No need to init here
test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)


In [103]:
params = {
   'iterations':1500,
   'loss_function': 'RMSE',
   'learning_rate':0.0109, #1 0.102,
   'depth': 6,
   'l2_leaf_reg': 6,
   
   'border_count': 7,
   'thread_count': 7,
   
   'bagging_temperature': 2,
   'random_strength': 2.23,
   'colsample_bylevel': 0.85,
   
   'custom_metric': ['MAPE', 'R2'], #  'MAPE'] # R2, Quantile,
   'eval_metric': 'R2', # 'MAPE', # Overfitting detection if `use_best_model` is enabled
   'random_seed': 42,
   
   # 'max_ctr_complexity': 2,
   #'logging_level': 'Silent',
   'use_best_model':False # Takes
}

In [104]:
reg_model = CatBoostRegressor(**params)

In [105]:
reg_model.fit(train_pool, eval_set=val_pool, plot=True, verbose=100)

0:	learn: 0.0056284	test: -1.1001253	best: -1.1001253 (0)	total: 11.6ms	remaining: 17.3s


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

100:	learn: 0.3176063	test: -1.0729547	best: -1.0716411 (88)	total: 450ms	remaining: 6.24s
200:	learn: 0.4148001	test: -1.0631020	best: -1.0614184 (185)	total: 754ms	remaining: 4.87s
300:	learn: 0.4615958	test: -1.0678930	best: -1.0612769 (210)	total: 1.07s	remaining: 4.28s
400:	learn: 0.4903113	test: -1.0641307	best: -1.0612769 (210)	total: 1.33s	remaining: 3.64s
500:	learn: 0.5117127	test: -1.0654602	best: -1.0612769 (210)	total: 1.59s	remaining: 3.16s
600:	learn: 0.5275189	test: -1.0673340	best: -1.0612769 (210)	total: 1.81s	remaining: 2.71s
700:	learn: 0.5464964	test: -1.0661995	best: -1.0612769 (210)	total: 2.18s	remaining: 2.49s
800:	learn: 0.5606561	test: -1.0643354	best: -1.0612769 (210)	total: 2.42s	remaining: 2.12s
900:	learn: 0.5770164	test: -1.0718586	best: -1.0612769 (210)	total: 2.71s	remaining: 1.8s
1000:	learn: 0.5905360	test: -1.0797002	best: -1.0612769 (210)	total: 3.08s	remaining: 1.53s
1100:	learn: 0.6018407	test: -1.0860271	best: -1.0612769 (210)	total: 3.39s	remai

In [ ]:
importances = reg_model.get_feature_importance(prettified=True)
importances

,Feature Id,Importances
0,Order Billing Country,42.582029
1,New Cat,37.843022
2,num mon,17.050671
3,year,2.524278


In [ ]:
X_test['predict_qty']=reg_model.predict(X_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X_test

,New Cat,num mon,year,Order Billing Country,predict_qty
454,Pants,2.0,2021.0,United States,2561.181215
590,T-Shirt,2.0,2021.0,Singapore,280.456674
780,Tops,2.0,2020.0,Italy,657.863006
631,T-Shirt,6.0,2020.0,South Korea,269.323698
1113,Underwear,9.0,2020.0,France,338.547116
...,...,...,...,...,...
359,Leggings,2.0,2021.0,France,845.606622
627,T-Shirt,6.0,2020.0,Israel,233.601698
567,T-Shirt,1.0,2020.0,South Korea,291.596044
56,Accessory,5.0,2020.0,Canada,345.956343


In [ ]:
y_test

,Total net sales
454,4635.00
590,450.00
780,575.01
631,500.00
1113,383.35
...,...
359,1078.32
627,125.00
567,150.00
56,95.00


In [ ]:
df2=X

In [ ]:
df2['year']=2021.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df2.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df2['Prediction_sale']=reg_model.predict(df2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df2['Prediction_sale']=df2['Prediction_sale'].apply(np.ceil)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from google.colab import files
df2.to_csv('webstore_prediction_sale.csv',index=False) 
files.download('webstore_prediction_sale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#money

In [ ]:
from google.colab import files
f1=files.upload()

Saving Webstore_Sales_Report_jermie_Table (2).csv to Webstore_Sales_Report_jermie_Table (2).csv


In [ ]:
df=pd.read_csv(r'Webstore_Sales_prediction.csv')
#df.apply(lambda x:len(x.unique()))